## Import

In [ ]:
# Chỉ dùng trên colab thì mới thêm dòng này vì dataset lưu trên drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import itertools

# Python lib used for data preprocessing
## for manipulating and processing tabular and numerical data.
import pandas as pd # Reading dataset, data manipulation and analysis
import numpy as np # Number computing
## for loading and preprocessing text data.
import json
import re

# Python lib used for reading dataframe (trong file outline là reading DATASET nhưng ko biết)
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
## Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
## MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
## for visualizing data and results
import matplotlib.pyplot as plt

In [ ]:
# Spotify library for connecting Spotify API
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## 1. Data Exploration/Preparation

Giải thích về việc import 2 file <br>
Vì thuật toán sẽ gợi ý bài hát dựa trên thể loại nhạc, độ phổ biến (popularity) và năm của bài hát (release_date). Nhưng với dataset track.csv thì không có cột genres nên đành sẽ phải nhập genres vào từ cột artists.csv

In [ ]:
# spotify_df lấy dữ liệu từ track.csv
spotify_df = pd.read_csv('/content/drive/MyDrive/Tài liệu/UEF/Đang học/Business Intelligence /Coding/Report/music_recommendation - Duy/dataset/tracks.csv')

In [ ]:
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [ ]:
spotify_df.dtypes

id                   object
name                 object
popularity            int64
duration_ms           int64
explicit              int64
artists              object
id_artists           object
release_date         object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object

In [ ]:
# data_w_genre lấy dữ liệu từ artists.csv
data_w_genre = pd.read_csv('/content/drive/MyDrive/Tài liệu/UEF/Đang học/Business Intelligence /Coding/Report/music_recommendation - Duy/dataset/artists.csv')
data_w_genre.tail()

,id,followers,genres,name,popularity
1162090,3cOzi726Iav1toV2LRVEjp,4831.0,['black comedy'],Ali Siddiq,34
1162091,6LogY6VMM3jgAE6fPzXeMl,46.0,[],Rodney Laney,2
1162092,19boQkDEIay9GaVAWkUhTa,257.0,[],Blake Wexler,10
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15
1162094,2bP2cNhNBdKXHC6AnqgyVp,40.0,['new comedy'],Gabe Kea,8


In [ ]:
data_w_genre.dtypes

id             object
followers     float64
genres         object
name           object
popularity      int64
dtype: object

### Xử lý dataframe "data_w_genre" vì python đang đọc Genres như là string chứ không phải là object

Đoạn này nó đọc như là một string nè

In [ ]:
data_w_genre['genres'].values[1200]

"['kleine hoerspiel', 'lesen']"

In [ ]:
data_w_genre['genres'].values[1200][0]

'['

Đoạn này nó đã đọc Genres như là list

In [ ]:
# regex statement to extract the genre and input into a list
data_w_genre['genres_upd'] = data_w_genre['genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", x)])

In [ ]:
data_w_genre['genres_upd'].values[1200][0]

'kleine_hoerspiel'

### Nhập cột genres từ data_w_genre vào spotify_df

Cột Genres đã sử dụng được và sẽ được nhập vào spotify_df dựa trên sự trùng khớp của artists giữa hai dataframe. Trước tiên là xử lý cột artists của spotify_df sang dạng list, giống như cách đã làm với cột genres ở trên

In [ ]:
spotify_df['artists_upd_v1'] = spotify_df['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [ ]:
spotify_df['artists'].values[0]

"['Uli']"

In [ ]:
spotify_df['artists_upd_v1'].values[0][0]

'Uli'

In [ ]:
spotify_df.head(5)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,...,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,[Uli]
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,...,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,[Fernando Pessoa]
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,...,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,[Ignacio Corsini]
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,...,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,[Ignacio Corsini]
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,...,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,[Dick Haymes]


In [ ]:
spotify_df[spotify_df['artists_upd_v1'].apply(lambda x: not x)].head(5)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1
164,1xEEYhWxT4WhDQdxfPCT8D,Snake Rag,20,194533,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.708,0.361,...,-11.764,0,0.0441,0.994,0.883,0.103,0.902,105.695,4,[]
170,3rauXVLOOM5BlxWqUcDpkg,Chimes Blues,14,170827,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.546,0.189,...,-15.984,1,0.0581,0.996,0.908,0.339,0.554,80.318,4,[]
172,1UdqHVRFYMZKU2Q7xkLtYc,Pickin' On Your Baby,11,197493,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.520,0.153,...,-14.042,1,0.0440,0.995,0.131,0.353,0.319,102.937,4,[]
174,0Vl2DO5U6FjgBpzCtBN3OA,Everybody Loves My Baby,10,152507,0,"[""Clarence Williams' Blue Five""]",['6RuQvIr0t0otZHnAxXTGkm'],1923,0.514,0.193,...,-13.920,0,0.2380,0.996,0.199,0.248,0.665,180.674,4,[]
180,5SvyP1ZeJX1jA7AOZD08NA,Tears,10,187227,0,"[""King Oliver's Creole Jazz Band""]",['08Zk65toyJllap1MnzljxZ'],1923,0.359,0.357,...,-11.810,1,0.0511,0.994,0.819,0.290,0.753,205.053,4,[]


In [ ]:
spotify_df['artists_upd_v2'] = spotify_df['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
spotify_df['artists_upd'] = np.where(spotify_df['artists_upd_v1'].apply(lambda x: not x), spotify_df['artists_upd_v2'], spotify_df['artists_upd_v1'] )

In [ ]:
spotify_df['artists_song'] = spotify_df.apply(lambda row: str(row['artists_upd'][0]) + str(row['name']), axis=1)

In [ ]:
spotify_df.sort_values(['artists_song','release_date'], ascending = False, inplace = True)

In [ ]:
# Check dứ liệu trùng lặp
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,...,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,...,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You
92524,1M4qEo4HE3PRaCOM7EXNJq,Adore You,74,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-06,0.676,0.771,...,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [ ]:
# Drop các dữ liệu trùng lặp
spotify_df.drop_duplicates('artists_song',inplace = True)

In [ ]:
spotify_df[spotify_df['name']=='Adore You']

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song
86217,5AnCLGg35ziFOloEnXK4uu,Adore You,71,278747,0,['Miley Cyrus'],['5YGY8feqx7naU7z4HrwZM6'],2013-10-04,0.583,0.655,...,0.1110,0.000004,0.113,0.201,119.759,4,[Miley Cyrus],[],[Miley Cyrus],Miley CyrusAdore You
91884,3jjujdWJ72nww5eGnfs2E7,Adore You,88,207133,0,['Harry Styles'],['6KImCVD70vtIoJWnq6nGn3'],2019-12-13,0.676,0.771,...,0.0237,0.000007,0.102,0.569,99.048,4,[Harry Styles],[],[Harry Styles],Harry StylesAdore You


In [ ]:
artists_exploded = spotify_df[['artists_upd','id']].explode('artists_upd')

In [ ]:
print(data_w_genre.columns)

Index(['id', 'followers', 'genres', 'name', 'popularity', 'genres_upd'], dtype='object')


In [ ]:
artists_exploded_enriched = artists_exploded.merge(data_w_genre, how = 'left', left_on = 'artists_upd',right_on = 'name')
artists_exploded_enriched_nonnull = artists_exploded_enriched[~artists_exploded_enriched.genres_upd.isnull()]

In [ ]:
artists_exploded_enriched_nonnull[artists_exploded_enriched_nonnull['id_x'] =='6KuQTIu1KoTTkLXKrwlLPV']

,artists_upd,id_x,id_y,followers,genres,name,popularity,genres_upd
195444,Robert Schumann,6KuQTIu1KoTTkLXKrwlLPV,2UqjDAXnDxejEyE0CzfUrZ,423826.0,"['classical', 'early romantic era', 'german ro...",Robert Schumann,64.0,"[classical, early_romantic_era, german_romanti..."
195445,Vladimir Horowitz,6KuQTIu1KoTTkLXKrwlLPV,4Ws5hSoABAwvGJ4LhHwHgq,92365.0,"['classical', 'classical performance', 'classi...",Vladimir Horowitz,54.0,"[classical, classical_performance, classical_p..."


In [ ]:
artists_genres_consolidated = artists_exploded_enriched_nonnull.groupby('id_x')['genres_upd'].apply(list).reset_index()

In [ ]:
artists_genres_consolidated['consolidates_genre_lists'] = artists_genres_consolidated['genres_upd'].apply(lambda x: list(set(list(itertools.chain.from_iterable(x)))))

In [ ]:
artists_genres_consolidated.head()

,id_x,genres_upd,consolidates_genre_lists
0,0004Uy71ku11n3LMpuyf59,[[polish_rock]],[polish_rock]
1,000CSYu4rvd8cQ7JilfxhZ,"[[country_quebecois, rock_quebecois]]","[rock_quebecois, country_quebecois]"
2,000DsoWJKHdaUmhgcnpr8j,[[barnmusik]],[barnmusik]
3,000G1xMMuwxNHmwVsBdtj1,"[[candy_pop, new_wave, new_wave_pop, permanent...","[new_wave, candy_pop, rock, power_pop, permane..."
4,000KblXP5csWFFFsD6smOy,"[[chamame, folclore_salteno, folklore_argentino]]","[chamame, folklore_argentino, folclore_salteno]"


In [ ]:
artists_genres_consolidated.reset_index(drop=True, inplace=True)
spotify_df.reset_index(drop=True, inplace=True)
print(spotify_df.index)
print(artists_genres_consolidated.index)

RangeIndex(start=0, stop=523475, step=1)
RangeIndex(start=0, stop=514222, step=1)


In [ ]:
spotify_df = spotify_df.merge(artists_genres_consolidated[['id_x', 'consolidates_genre_lists']], left_on='id', right_on='id_x', how='left')

In [ ]:
spotify_df.head(20)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,id_x,consolidates_genre_lists
0,3u1C6nWVRoP5F0w8gGrDL3,사랑의 미로,25,222380,0,['최진희'],['1NSrAf8XJYJVgAXKoxaMet'],1987-06-01,0.367,0.1940,...,0.1620,0.3670,144.316,4,[최진희],[],[최진희],최진희사랑의 미로,3u1C6nWVRoP5F0w8gGrDL3,[trot]
1,1Mv4u308L16NZDZiD6HZCy,사랑은 힘든가봐,28,213440,0,['지수'],['4c9QIMfEbIIynuaswyxGx9'],2005-12-23,0.675,0.7850,...,0.3530,0.6230,103.008,4,[지수],[],[지수],지수사랑은 힘든가봐,NaN,NaN
2,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.3410,...,0.1440,0.2940,135.667,4,[지선],[],[지선],지선어떡하죠,1jvoY322nxyKXq8OBhgmSY,[]
3,2ghebdwe2pNXT4eL34T7pW,그아픔까지사랑한거야,32,237688,0,['조정현'],['2WTpsPucygbYRnCnoEUkJQ'],1989-06-15,0.447,0.2150,...,0.0649,0.1770,71.979,4,[조정현],[],[조정현],조정현그아픔까지사랑한거야,NaN,NaN
4,7rxpWwcXNgDUXl0wN0gUvp,천국의 기억 장정우 Version,31,280372,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.494,0.6560,...,0.1110,0.4200,82.003,4,[장정우],[],[장정우],장정우천국의 기억 장정우 Version,7rxpWwcXNgDUXl0wN0gUvp,[]
5,0cEvzbXjxkOxgBUmBUcHZW,그것만은..,32,294452,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.311,0.4870,...,0.2410,0.1590,59.464,4,[장정우],[],[장정우],장정우그것만은..,0cEvzbXjxkOxgBUmBUcHZW,[]
6,58pl1flp2tyrONFUCMAZrc,"쇼팽, 피아노 협주곡 1번 Op.11 E 단조 - 2악장 로만차",26,219716,0,['이소아'],['2voJP621yvNsZAZWgzd9pS'],2003-12-24,0.417,0.0948,...,0.1090,0.2550,80.106,4,[이소아],[],[이소아],"이소아쇼팽, 피아노 협주곡 1번 Op.11 E 단조 - 2악장 로만차",NaN,NaN
7,2xnE8SFDOs20FTAaVl98nr,처음 그 자리에,40,233213,0,['이보람'],['151APvoui0QiXyOakfPGt5'],2007-11-09,0.524,0.6020,...,0.1060,0.1550,139.788,4,[이보람],[],[이보람],이보람처음 그 자리에,NaN,NaN
8,6uRXQW8BqB3N9WKfe9gfdw,Title 허밍,37,102307,0,['이경섭'],['191huMISbbIeUELiiEGZ7L'],2007-11-09,0.825,0.6040,...,0.0875,0.8980,98.272,4,[이경섭],[],[이경섭],이경섭Title 허밍,6uRXQW8BqB3N9WKfe9gfdw,[]
9,02q7sJH0d1lC0oDd8gf513,별,44,302022,0,['유미'],['0kgK64SIuInLUon0mn9o4h'],2006-12-12,0.299,0.3240,...,0.1560,0.0767,79.594,4,[유미],[],[유미],유미별,02q7sJH0d1lC0oDd8gf513,[]


## 2. Feature Engineering

### - Normalize float variables
### - OHE Year and Popularity Variables
### - Create TF-IDF features off of artist genres

In [ ]:
spotify_df.dtypes

id                           object
name                         object
popularity                    int64
duration_ms                   int64
explicit                      int64
artists                      object
id_artists                   object
release_date                 object
danceability                float64
energy                      float64
key                           int64
loudness                    float64
mode                          int64
speechiness                 float64
acousticness                float64
instrumentalness            float64
liveness                    float64
valence                     float64
tempo                       float64
time_signature                int64
artists_upd_v1               object
artists_upd_v2               object
artists_upd                  object
artists_song                 object
id_x                         object
consolidates_genre_lists     object
dtype: object

In [ ]:
# Normalize float variables
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index.values

In [ ]:
float_cols

array(['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype=object)

### 2.1. OHE Popularity and Year Variables

#### Popularity

In [ ]:
spotify_df['popularity'].describe()

count    523475.000000
mean         27.235870
std          18.030233
min           0.000000
25%          13.000000
50%          27.000000
75%          40.000000
max          99.000000
Name: popularity, dtype: float64

In [ ]:
# create 5 point buckets for popularity
spotify_df['popularity_red'] = spotify_df['popularity'].apply(lambda x: int(x/5))

In [ ]:

spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,id_x,consolidates_genre_lists,popularity_red
0,3u1C6nWVRoP5F0w8gGrDL3,사랑의 미로,25,222380,0,['최진희'],['1NSrAf8XJYJVgAXKoxaMet'],1987-06-01,0.367,0.194,...,0.367,144.316,4,[최진희],[],[최진희],최진희사랑의 미로,3u1C6nWVRoP5F0w8gGrDL3,[trot],5
1,1Mv4u308L16NZDZiD6HZCy,사랑은 힘든가봐,28,213440,0,['지수'],['4c9QIMfEbIIynuaswyxGx9'],2005-12-23,0.675,0.785,...,0.623,103.008,4,[지수],[],[지수],지수사랑은 힘든가봐,NaN,NaN,5
2,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.341,...,0.294,135.667,4,[지선],[],[지선],지선어떡하죠,1jvoY322nxyKXq8OBhgmSY,[],8
3,2ghebdwe2pNXT4eL34T7pW,그아픔까지사랑한거야,32,237688,0,['조정현'],['2WTpsPucygbYRnCnoEUkJQ'],1989-06-15,0.447,0.215,...,0.177,71.979,4,[조정현],[],[조정현],조정현그아픔까지사랑한거야,NaN,NaN,6
4,7rxpWwcXNgDUXl0wN0gUvp,천국의 기억 장정우 Version,31,280372,0,['장정우'],['5L7zKs2ftwENWOMI7LFaN1'],2003-12-24,0.494,0.656,...,0.420,82.003,4,[장정우],[],[장정우],장정우천국의 기억 장정우 Version,7rxpWwcXNgDUXl0wN0gUvp,[],6


#### Year

In [ ]:
spotify_df['year'] = spotify_df['release_date'].apply(lambda x: x.split('-')[0])

In [ ]:
# Convert 'year' column to integers
spotify_df['year'] = spotify_df['year'].astype(int)

# Assuming 'year' column contains the year information
condition = (spotify_df['year'] >= 1900) & (spotify_df['year'] <= 2009)

# Filter out records within the specified range
filtered_df = spotify_df[~condition]

# If you want to modify the original DataFrame, you can reassign the filtered DataFrame back to it
spotify_df = filtered_df.copy()


In [ ]:
spotify_df[spotify_df['year']==1925]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,id_x,consolidates_genre_lists,popularity_red,year


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
spotify_df.head(20)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,id_x,consolidates_genre_lists,popularity_red,year
2,1jvoY322nxyKXq8OBhgmSY,어떡하죠,44,244360,0,['지선'],['2Mo9NQaNCFCWSR5CnlfmbN'],2011-10-13,0.606,0.341,0,-7.094,1,0.0513,0.779000,0.000000,0.1440,0.294,135.667,4,[지선],[],[지선],지선어떡하죠,1jvoY322nxyKXq8OBhgmSY,[],8,2011
11,2OCsS6p88T7Tba37LbwFoG,아쉬운 마음인걸,45,188973,0,['에이 스타일'],['6Isq5QAXFUqu6WkFcEAyF1'],2011-10-13,0.708,0.700,3,-5.707,0,0.0318,0.457000,0.000000,0.1470,0.902,150.048,4,[에이 스타일],[],[에이 스타일],에이 스타일아쉬운 마음인걸,2OCsS6p88T7Tba37LbwFoG,[],9,2011
20,2w5L5kQVkSRdduNewYPDA5,My Island Home,20,184120,0,['龔詩嘉'],['3YJXJABOwuadyp7MPN9wbi'],2015-07-31,0.450,0.782,0,-5.905,1,0.0306,0.134000,0.000000,0.0632,0.475,87.980,4,[龔詩嘉],[],[龔詩嘉],龔詩嘉My Island Home,2w5L5kQVkSRdduNewYPDA5,"[singaporean_pop, singaporean_mandopop]",4,2015
38,72nxG9oS1W6lta9ZqIjXYe,Magic∞world,35,247307,0,['黒崎真音'],['4SLTgwsFXbomwbNjsAvs3E'],2011-11-30,0.473,0.947,4,-4.398,0,0.1170,0.000258,0.000097,0.0951,0.387,171.018,4,[黒崎真音],[],[黒崎真音],黒崎真音Magic∞world,72nxG9oS1W6lta9ZqIjXYe,"[anime, seiyu]",7,2011
69,4JLOiMe4cTaquDofw2K7Dh,还可以爱吗,34,295890,0,['黄霄雲'],['3j9vrMIrCGgFmq0276a3kg'],2013,0.465,0.510,11,-8.443,1,0.0307,0.674000,0.000734,0.1470,0.192,146.033,4,[黄霄雲],[],[黄霄雲],黄霄雲还可以爱吗,4JLOiMe4cTaquDofw2K7Dh,[],6,2013
70,47O6EPP78iQOOma2OVsYwS,星辰大海,61,207857,0,['黄霄雲'],['4vcc1VAbvzJjbMTsPYL6OU'],2021-01-15,0.303,0.550,4,-6.855,1,0.0333,0.322000,0.000000,0.1010,0.161,167.874,4,[黄霄雲],[],[黄霄雲],黄霄雲星辰大海,47O6EPP78iQOOma2OVsYwS,[],12,2021
71,6Bp1c1NbLGFkkl8kkUiWw3,風月,40,228590,0,['黃齡'],['6OAQlHABRwPJYVTkaSmiJ9'],2015-11-16,0.513,0.512,0,-6.420,1,0.0366,0.514000,0.000000,0.1080,0.289,123.865,4,[黃齡],[],[黃齡],黃齡風月,6Bp1c1NbLGFkkl8kkUiWw3,"[chinese_drama_ost, mainland_chinese_pop]",8,2015
137,0dQ3oA4k4AhwXlCjoDdqU1,無神論 - 中文版,35,197227,0,['黃立行'],['4ccMWuTuYspEkm2gh7KrkO'],2015-02-06,0.585,0.495,10,-9.428,0,0.0328,0.114000,0.002950,0.1920,0.379,120.047,4,[黃立行],[],[黃立行],黃立行無神論 - 中文版,0dQ3oA4k4AhwXlCjoDdqU1,"[mandopop, taiwan_pop]",7,2015
159,0W0iJPVqxnNKmQEXVGGlmk,天地,24,299337,0,['黃文星'],['4Tttk6v0QiiPXMTZxzWtPY'],2010-07-06,0.526,0.374,2,-9.521,1,0.0254,0.400000,0.000007,0.2410,0.221,94.878,4,[黃文星],[],[黃文星],黃文星天地,0W0iJPVqxnNKmQEXVGGlmk,"[taiwanese_pop, hokkien_pop]",4,2010
160,798BSvqw7wOL6HDboCWDR1,三心兩意 (feat. 曾昱嘉),34,271778,0,"['黃文星', '曾昱嘉']","['4Tttk6v0QiiPXMTZxzWtPY', '1pgiuGaVNIqGee2xUh...",2010-07-06,0.614,0.484,0,-8.611,1,0.0311,0.209000,0.000000,0.1740,0.219,128.032,4,"[黃文星, 曾昱嘉]",[],"[黃文星, 曾昱嘉]",黃文星三心兩意 (feat. 曾昱嘉),798BSvqw7wOL6HDboCWDR1,"[taiwanese_pop, hokkien_pop, taiwan_pop]",6,2010


In [ ]:
spotify_df['year'] = spotify_df['year'].astype(str)
spotify_df[spotify_df['year']=="2006"]

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,id_x,consolidates_genre_lists,popularity_red,year


In [ ]:
# Selecting 3 out of 5 attributes
subset_attributes = ['name', 'consolidates_genre_lists']
subset_df = spotify_df[subset_attributes]
subset_df.head(20)

,name,consolidates_genre_lists
2,어떡하죠,[]
11,아쉬운 마음인걸,[]
20,My Island Home,"[singaporean_pop, singaporean_mandopop]"
38,Magic∞world,"[anime, seiyu]"
69,还可以爱吗,[]
70,星辰大海,[]
71,風月,"[chinese_drama_ost, mainland_chinese_pop]"
137,無神論 - 中文版,"[mandopop, taiwan_pop]"
159,天地,"[taiwanese_pop, hokkien_pop]"
160,三心兩意 (feat. 曾昱嘉),"[taiwanese_pop, hokkien_pop, taiwan_pop]"


#### Create OHE features

In [ ]:
#simple function to create OHE features
#this gets passed later on
def ohe_prep(df, column, new_name):
    """
    Create One Hot Encoded features of a specific column

    Parameters:
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        new_name (str): new column name to be used

    Returns:
        tf_df: One hot encoded features
    """

    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)
    return tf_df

### 2.2. Create TF-IDF features off of artist genres

In [ ]:
# tfidf can't handle nulls so fill any null values with an empty list
spotify_df['consolidates_genre_lists'] = spotify_df['consolidates_genre_lists'].apply(lambda d: d if isinstance(d, list) else [])

In [ ]:
#function to build entire feature set
def create_feature_set(df, float_cols):
    """
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters:
        df (pandas dataframe): Spotify Dataframe
        float_cols (list(str)): List of float columns that will be scaled

    Returns:
        final: final set of features
    """

    #tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['consolidates_genre_lists'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre|' + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop=True, inplace=True)

    #ohe year and popularity
    year_ohe = ohe_prep(df, 'year','year') * 0.5
    popularity_ohe = ohe_prep(df, 'popularity_red','pop') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)

    #add song id
    final['id']=df['id'].values

    return final

In [ ]:
complete_feature_set = create_feature_set(spotify_df, float_cols=float_cols)

In [ ]:
complete_feature_set.tail(20)

Output hidden; open in https://colab.research.google.com to view.

## 3. Connect to Spotify API

In [ ]:
#client id and secret for application
client_id = 'ca9d19a6509940eea3a48859573112bf'
client_secret= '5121914d1edc497db32b93a46dd9ab89'

In [ ]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
from spotipy.oauth2 import SpotifyOAuth
import webbrowser

# Replace these variables with your own data
client_id = 'ca9d19a6509940eea3a48859573112bf'
client_secret = '5121914d1edc497db32b93a46dd9ab89'
redirect_uri = 'http://localhost:8080/callback'
#scope = 'YOUR_DESIRED_SCOPE'  # e.g., 'playlist-read-private'

# Initialize SpotifyOAuth object
auth_manager = SpotifyOAuth(client_id=client_id,
                            client_secret=client_secret,
                            redirect_uri=redirect_uri,
                            scope=scope)

# Get the authorization URL
auth_url = auth_manager.get_authorize_url()

# Open the authorization URL in the browser, you can copy the URL if the browser doesn't open
print("Please navigate to this URL to authorize:", auth_url)
webbrowser.open(auth_url)

# After authorization, Spotify redirects you to the redirect_uri with a code in the URL
# You need to manually copy this URL from your browser and paste it here
response_url = input("Paste the URL you were redirected to here: ")

# Extract the code from the URL
code = auth_manager.parse_response_code(response_url)

# Use the code to get the token
token = auth_manager.get_access_token(code, as_dict=False)

print("Your access token is:", token)
sp = spotipy.Spotify(auth=token)

Please navigate to this URL to authorize: https://accounts.spotify.com/authorize?client_id=ca9d19a6509940eea3a48859573112bf&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2Fcallback&scope=user-library-read
Paste the URL you were redirected to here: http://localhost:8080/callback?code=AQBjjGDry3uRJ2nRnIzj_lbgOCgJj1qwW5fmEHeiZjD0B8SdsBdiPWrjBZPkuEa-EbK474dNJk5E3w1DDi8Tgn4wrS9mOYEijCBD-p9q343pZA7-dWGub2GS996BpOU9jTF5uVI_ZVoMVpd7tDsKR7wtrXUJkhSBXId0a9jF322X7RDtdZSy-WYWFchzdeDr9ZuSMeg
Your access token is: BQD07wXIPAWQhJ6RTDUqZFuDubPPg58NVTBPM6bCJv1eEYBiSGSI1-neu0hgoJkLm1pHs2YSGjRxt8bQrLwmhP9wZJRHMHh10b3pIRAuD_qGYex5PzouAn5g5DdXc_eTrq5CBlvOGH_1O7W2Peslw_7dPtmzh8H2enYnI-efUaOXPI9eKw43siB-zfVhWFTyFWv34BuIFgH-S6uUgAw


In [ ]:
#gather playlist names and images.
#images aren't going to be used until I start building a UI
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
id_name

{'Sad melody, r&b, pop': '4CWCzaBhn1ClWNvbujw0rT',
 'US-UK, Rap, Gangster rap, R&B': '4AAXDU2g2WSJIOokMOKV1L',
 'VN, JP, US-UK, Rap, Pop, Melody Rap, R&B': '0k21jLsZ1P1xzfqJNVr2E9',
 'JP, pop, city pop': '7eiKgDAznmGRsiYQljvJz3',
 'Rap, gangster rap, hardstyle, phonk': '0MdceVMwj1J419z6NKaIFH',
 'Pop, r&b, rap, melody rap': '0xCrOwDEhYS5HDTOFdsE93',
 'US-UK, VN, JP, Pop, R&B, Melody rap': '0ldrnai67hsboIxqCu8vy3',
 'Pop, melody rap, r&b, classical': '5isL3r6JZFRtgVSZwuw33K',
 'VN, Rap, R&B, Melody rap': '0ZpahEca1VrAB9dBjQ0s89'}

In [ ]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """
    Pull songs from a specific playlist.

    Parameters:
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify dataframe

    Returns:
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """

    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])

    # Updates the playlist_name DataFrame to keep only the rows where its ids match those in df, and then sorts these filtered rows by the 'date_added'
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)

    return playlist

In [ ]:
id_name

{'Sad melody, r&b, pop': '4CWCzaBhn1ClWNvbujw0rT',
 'US-UK, Rap, Gangster rap, R&B': '4AAXDU2g2WSJIOokMOKV1L',
 'VN, JP, US-UK, Rap, Pop, Melody Rap, R&B': '0k21jLsZ1P1xzfqJNVr2E9',
 'JP, pop, city pop': '7eiKgDAznmGRsiYQljvJz3',
 'Rap, gangster rap, hardstyle, phonk': '0MdceVMwj1J419z6NKaIFH',
 'Pop, r&b, rap, melody rap': '0xCrOwDEhYS5HDTOFdsE93',
 'US-UK, VN, JP, Pop, R&B, Melody rap': '0ldrnai67hsboIxqCu8vy3',
 'Pop, melody rap, r&b, classical': '5isL3r6JZFRtgVSZwuw33K',
 'VN, Rap, R&B, Melody rap': '0ZpahEca1VrAB9dBjQ0s89'}

Demo playlist <br>
US-UK, Rap, Gangster rap, R&B <br>
Pop, melody rap, r&b, classical <br>
US-UK, VN, JP, Pop, R&B, Melody rap <br>
Rap, gangster rap, hardstyle, phonk <br>
Sad melody, r&b, pop <br>

In [ ]:
choosen_playlist = 'US-UK, Rap, Gangster rap, R&B'
my_choosen_playlist = create_necessary_outputs(choosen_playlist, id_name, spotify_df)
#playlist_chill = create_necessary_outputs('chill',id_name, spotify_df)
#playlist_classical = create_necessary_outputs('Epic Classical',id_name, spotify_df)

In [ ]:
import math
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):
    """
    Visualize cover art of the songs in the inputted dataframe

    Parameters:
        df (pandas dataframe): Playlist Dataframe
    """

    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5

    for i, url in enumerate(temp):
        plt.subplot(math.ceil(len(temp) / columns), columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color='w', fontsize=0.1)
        plt.yticks(color='w', fontsize=0.1)
        plt.xlabel(df['name'].values[i], fontsize=12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
my_choosen_playlist

,artist,name,id,url,date_added
3,The Weeknd,Sidewalks,4h90qkbnW1Qq6pBhoPvwko,https://i.scdn.co/image/ab67616d00001e024718e2...,2024-03-17 06:38:48+00:00
50,The Weeknd,Starboy,7MXVkk9YMctZqd1Srtv4MB,https://i.scdn.co/image/ab67616d00001e024718e2...,2024-03-17 06:38:48+00:00
59,twocolors,Lovefool,51Of5p3lKZeOg6itfs4og4,https://i.scdn.co/image/ab67616d00001e02985af3...,2024-03-17 06:38:48+00:00
64,Playboi Carti,Sky,29TPjc8wxfz4XMn21O7VsZ,https://i.scdn.co/image/ab67616d00001e0298ea0e...,2024-03-17 06:38:48+00:00
65,French Montana,Unforgettable,3B54sVLJ402zGa6Xm4YGNe,https://i.scdn.co/image/ab67616d00001e028a3119...,2024-03-17 06:38:48+00:00
66,Don Toliver,No Idea,7AzlLxHn24DxjgQX73F9fU,https://i.scdn.co/image/ab67616d00001e0245190a...,2024-03-17 06:38:48+00:00
70,The Weeknd,Pray For Me (with Kendrick Lamar),77UjLW8j5UAGAGVGhR5oUK,https://i.scdn.co/image/ab67616d00001e02c027ad...,2024-03-17 06:38:48+00:00
71,ScHoolboy Q,X (with 2 Chainz & Saudi),4LmAnpjlhWTahvRkYR8xJa,https://i.scdn.co/image/ab67616d00001e02c027ad...,2024-03-17 06:38:48+00:00
72,Lil Wayne,Love Me,2XHzzp1j4IfTNp1FTn7YFg,https://i.scdn.co/image/ab67616d00001e0280c64a...,2024-03-17 06:38:48+00:00
73,21 Savage,Glock In My Lap,6pcywuOeGGWeOQzdUyti6k,https://i.scdn.co/image/ab67616d00001e02aa5790...,2024-03-17 06:38:48+00:00


In [ ]:
visualize_songs(my_choosen_playlist)

Output hidden; open in https://colab.research.google.com to view.

# 4. Create Playlist Vector

In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """
    Summarize a user's playlist into a single vector

    Parameters:
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1.

    Returns:
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe):
    """

    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)

    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]

    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)

    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))

    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']

    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
complete_feature_set_myplaylist_vector, complete_feature_set_nonmyplaylist = generate_playlist_feature(complete_feature_set, my_choosen_playlist, 1.09)

In [ ]:
complete_feature_set_myplaylist_vector

genre|21st_century_classical    0.000000
genre|432hz                     0.000000
genre|48g                       0.000000
genre|_brasileira               0.000000
genre|_hip_hop                  0.435671
                                  ...   
year|2017                       1.500000
year|2018                       3.000000
year|2019                       2.500000
year|2020                       4.500000
year|2021                       0.000000
Length: 4010, dtype: float64

In [ ]:
complete_feature_set_nonmyplaylist

Output hidden; open in https://colab.research.google.com to view.

# 5. Generate Recommendations

Thuật toán Cosine Similarity

---



In [ ]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """
    Pull songs from a specific playlist.

    Parameters:
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist

    Returns:
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """

    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])

    return non_playlist_df_top_40

In [ ]:
rcm_top = generate_playlist_recos(spotify_df, complete_feature_set_myplaylist_vector, complete_feature_set_nonmyplaylist)

In [ ]:
rcm_top

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artists_song,id_x,consolidates_genre_lists,popularity_red,year,sim,url
362979,7EiZI6JVHllARrX9PUvAdX,Low Life (feat. The Weeknd),75,313547,1,"['Future', 'The Weeknd']","['1RyvyyTE3xzB2ZywiAwp0i', '1Xyo4u8uXC1ZmMpatF...",2016-04-13,0.722,0.331,8,-7.789,1,0.0725,0.33700,0.283000,0.1460,0.1020,143.974,4,"[Future, The Weeknd]",[],"[Future, The Weeknd]",FutureLow Life (feat. The Weeknd),7EiZI6JVHllARrX9PUvAdX,"[atl_hip_hop, southern_hip_hop, trap, canadian...",15,2016,0.795707,https://i.scdn.co/image/ab67616d00001e02626745...
60744,0NWqNXBJTpXbkI5rPWNy3p,All I Know,65,321027,1,"['The Weeknd', 'Future']","['1Xyo4u8uXC1ZmMpatF05PJ', '1RyvyyTE3xzB2ZywiA...",2016-11-25,0.467,0.417,8,-10.614,0,0.0740,0.42100,0.000202,0.1020,0.1710,120.021,4,"[The Weeknd, Future]",[],"[The Weeknd, Future]",The WeekndAll I Know,0NWqNXBJTpXbkI5rPWNy3p,"[atl_hip_hop, southern_hip_hop, trap, canadian...",13,2016,0.791206,https://i.scdn.co/image/ab67616d00001e024718e2...
60694,36YCdzT57us0LhDmCYtrNE,Rockin’,68,232880,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2016-11-25,0.738,0.885,9,-5.547,0,0.2060,0.01330,0.000000,0.0821,0.7520,112.878,4,[The Weeknd],[],[The Weeknd],The WeekndRockin’,36YCdzT57us0LhDmCYtrNE,"[canadian_contemporary_r&b, pop, canadian_pop]",13,2016,0.737039,https://i.scdn.co/image/ab67616d00001e024718e2...
60711,27BgDmciSjoxTG0almHTpZ,Love To Lay,66,223293,0,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2016-11-25,0.711,0.807,7,-5.052,0,0.0608,0.05200,0.000000,0.2140,0.6980,116.971,4,[The Weeknd],[],[The Weeknd],The WeekndLove To Lay,27BgDmciSjoxTG0almHTpZ,"[canadian_contemporary_r&b, pop, canadian_pop]",13,2016,0.736271,https://i.scdn.co/image/ab67616d00001e024718e2...
60747,0ifooM33X1iBxVYyOkSbXW,A Lonely Night,68,220173,0,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2016-11-25,0.853,0.663,7,-6.955,0,0.0755,0.04310,0.000009,0.0891,0.8640,113.998,4,[The Weeknd],[],[The Weeknd],The WeekndA Lonely Night,0ifooM33X1iBxVYyOkSbXW,"[canadian_contemporary_r&b, pop, canadian_pop]",13,2016,0.735643,https://i.scdn.co/image/ab67616d00001e024718e2...
124790,4cbdPT6uaBOgOQe3fLMofl,Power is Power (feat. The Weeknd & Travis Scott),68,211267,0,"['SZA', 'The Weeknd', 'Travis Scott', 'Game of...","['7tYKF4w9nC0nq9CsPZTHyP', '1Xyo4u8uXC1ZmMpatF...",2019-04-26,0.507,0.644,4,-7.485,1,0.0641,0.00413,0.000000,0.2170,0.1720,179.913,4,"[SZA, The Weeknd, Travis Scott, Game of Thrones]",[],"[SZA, The Weeknd, Travis Scott, Game of Thrones]",SZAPower is Power (feat. The Weeknd & Travis S...,4cbdPT6uaBOgOQe3fLMofl,"[r&b, canadian_contemporary_r&b, pop, canadian...",13,2019,0.730004,https://i.scdn.co/image/ab67616d00001e02d4024f...
60728,4oeaIftdpT3JuZLcCkKmVE,False Alarm,70,220307,0,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2016-11-25,0.517,0.677,8,-6.373,0,0.0746,0.03240,0.000034,0.1890,0.1920,173.978,4,[The Weeknd],[],[The Weeknd],The WeekndFalse Alarm,4oeaIftdpT3JuZLcCkKmVE,"[canadian_contemporary_r&b, pop, canadian_pop]",14,2016,0.728857,https://i.scdn.co/image/ab67616d00001e024718e2...
60741,6pDVjCUA3B1vg9waKutAsv,Attention,66,197653,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2016-11-25,0.671,0.428,4,-7.052,0,0.1240,0.68800,0.000182,0.1190,0.3620,148.040,4,[The Weeknd],[],[The Weeknd],The WeekndAttention,6pDVjCUA3B1vg9waKutAsv,"[canadian_contemporary_r&b, pop, canadian_pop]",13,2016,0.727577,https://i.scdn.co/image/ab67616d00001e024718e2...
60705,15k2nBQJ0teDmPZHrOXL2N,Ordinary Life,66,221920,0,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2016-11-25,0.537,0.587,7,-7.486,0,0.0771,0.02750,0.000000,0.0961,0.3650,149.748,4,[The Weeknd],[],[The Weeknd],The WeekndOrdinary Life,15k2nBQJ0teDmPZHrOXL2N,"[canadian_contemporary_r&b, pop, canadian_pop]",13,2016,0.724124,https://i.scdn.co/image/ab67616d00001e024718e2...
60673,1oGdVdYjeQvojGKDddxLQQ,True Colors

In [ ]:
visualize_songs(rcm_top)

Output hidden; open in https://colab.research.google.com to view.